# Simulated Annealing

In [ ]:
import random
import math

import numpy as np

In [ ]:
population = lambda items, s: np.array([random.randint(0, s) for _ in items])

In [ ]:
def evaluate(items, ind, weights = [-1, 1], max_weight = 65):
    p = sum(ind * items[:, 0])
    w = sum(ind * items[:, 1])

    r = [p, w] if w <= max_weight and w > 0 else [0, 1000]

    return (sum(np.array(weights) * r), r)    

In [ ]:
def annealing(items, s = 3, t = 10000.0, cool_rate = 0.995, min_t = 0.1, step = 1, max_weight = 65):
    p = population(items, s)
    
    while t > min_t:
        idx = random.randint(0, len(p) - 1)
        
        q = p.copy()
        
        q[idx] = max(0, min(s, q[idx] + random.randint(-step, step)))

        pe, _ = evaluate(items, p, max_weight = max_weight)
        qe, _ = evaluate(items, q, max_weight = max_weight)

        prob = math.exp(-abs(qe - pe) / t)

        if qe < pe or random.random() < prob:
            p = q
        
        t *= cool_rate

    return p

In [ ]:
def annealing2(items, s = 3, t = 10000.0, cool_rate = 0.995, min_t = 0.1, max_weight = 65):
    p = np.zeros(len(items))
    
    exclude = lambda xs, x: xs[xs != x] 
    
    while t > min_t:
        idx = random.randint(0, len(p) - 1)
        
        q = p.copy()

        q[idx] = abs(q[idx] - random.randint(1, s))

        if q[idx] > p[idx]:
            for i in np.append(exclude(np.random.permutation(len(items)), idx), idx):
                w_over = sum(q * items[:, 1]) - max_weight
                    
                if w_over <= 0:
                    break
                else:
                    n = math.ceil(w_over / items[i, 1])
                    q[i] -= min(q[i], n)

        pe = sum(p * items[:, 0])
        qe = sum(q * items[:, 0])

        prob = math.exp(-abs(qe - pe) / t)

        if qe > pe or random.random() < prob:
            p = q

        t *= cool_rate

    return p

In [ ]:
def annealing3(items, n = 5, s = 3, t = 10000.0, cool_rate = 0.995, min_t = 0.1, max_weight = 65):
    p = np.zeros(len(items))
    
    exclude = lambda xs, x: xs[xs != x]
    
    while t > min_t:
        for _ in range(n):
            idx = random.randint(0, len(p) - 1)
        
            q = p.copy()
            
            q[idx] = abs(q[idx] - random.randint(1, s))

            random_ids = exclude(np.random.permutation(len(items)), idx)

            if q[idx] >= p[idx]:
                for i in np.append(random_ids, idx):
                    w_over = sum(q * items[:, 1]) - max_weight
                    
                    if w_over <= 0:
                        break
                    elif q[i] > 0:
                        n = math.ceil(w_over / items[i, 1])
                        q[i] -= min(q[i], n)
            else:
                for i in random_ids:
                    if q[i] < s and items[i, 1] <= max_weight - sum(q * items[:, 1]):
                        q[i] += 1
                        break

            pe = sum(p * items[:, 0])
            qe = sum(q * items[:, 0])

            prob = math.exp(-abs(qe - pe) / t)

            if qe > pe or random.random() < prob:
                p = q
        
        t *= cool_rate

    return p

In [ ]:
items1 = np.array([
    [120, 10],
    [130, 12],
    [80, 7],
    [100, 9],
    [250, 21],
    [185, 16]
])

In [ ]:
r = annealing(items1, 2)
print(f'{r}, {evaluate(items1, r)}')

In [ ]:
r2 = annealing2(items1)
print(f'{r2}, {evaluate(items1, r2)}')

In [ ]:
r3 = annealing3(items1)
print(f'{r3}, {evaluate(items1, r3)}')

In [ ]:
import collections

collections.Counter([tuple(annealing(items1)) for _ in range(100)])

In [ ]:
collections.Counter([tuple(annealing2(items1)) for _ in range(100)])

In [ ]:
collections.Counter([tuple(annealing3(items1)) for _ in range(100)])